In [25]:
import numpy as np
from math import log

def createDataSet():
    group = [[1.0, 1.2, 'A'], [1.1, 1.0, 'A'], [0.1, 0.2, 'B'], [0.0, 0.1, 'B']]
    labels = ['A', 'A', 'B', 'B']
    return group, labels

# 计算信息熵
def calcShannonEnt(dataSet):
    count = len(dataSet)
    labelsCount = {}
    for featVec in dataSet:
        print(featVec)
        feat = featVec[-1]
        if feat not in labelsCount.keys():
            labelsCount[feat] = 0
        labelsCount[feat] += 1
    result = 0.0
    for key in labelsCount:
        prop = float(labelsCount[key] / count)
        result += (-prop)*log(prop,2)
    return result

group, labels = createDataSet()
print(group[1])
shannonEnt = calcShannonEnt(group)
print(shannonEnt)

def splitDataSet(dataSet, index, value):
    result = []
    for featVec in dataSet:
        print(featVec)
        if featVec[index] == value:
            print("hello")
            tmpVec = featVec[:index]
            tmpVec.extend(featVec[index+1:])
            result.append(tmpVec)
    return result

splitDataSet = splitDataSet(group, 1, 1.0)
print(splitDataSet)

NameError: name 'array' is not defined